In [1]:
import config
import concurrent.futures
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from tqdm import tqdm

In [2]:
def findBordersHorizontal(img, border, side):
    if side == "l2r":
        for i in range(border):
            if np.sum(img[:,i]) > 0:
                return i
    elif side == "r2l":
        for i in range(img.shape[1] - 1, border, -1):
            if np.sum(img[:,i]) > 0:
                return i
    else:
        raise ValueError
    return border

def findBordersVertical(img, border, side):
    if side == "t2b":
        for i in range(border):
            if np.sum(img[i]) > 0:
                return i
    elif side == "b2t":
        for i in range(img.shape[0] - 1, border, -1):
            if np.sum(img[i]) > 0:
                return i
    else:
        raise ValueError
    return border


def findBorders(img, bordLeft, bordRight, bordTop, bordBottom):
    bordLeft = findBordersHorizontal(img, bordLeft, "l2r")
    bordRight = findBordersHorizontal(img, bordRight, "r2l")
    bordTop = findBordersVertical(img, bordTop, "t2b")
    bordBottom = findBordersVertical(img, bordBottom, "b2t")
    return bordLeft, bordRight, bordTop, bordBottom


In [3]:
# Function to find borders for a single image
def find_borders(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    return findBorders(img, img.shape[0] - 1, 0, img.shape[1] - 1, 0)

# Function to merge borders from multiple images
def merge_borders(borders):
    bordLeft, bordRight, bordTop, bordBottom = borders[0]
    for b in borders[1:]:
        bordLeftTmp, bordRightTmp, bordTopTmp, bordBottomTmp = b
        bordLeft = min(bordLeft, bordLeftTmp)
        bordRight = max(bordRight, bordRightTmp)
        bordTop = min(bordTop, bordTopTmp)
        bordBottom = max(bordBottom, bordBottomTmp)
    return bordLeft, bordRight, bordTop, bordBottom

# path to the folder with already segmented data
path_folder = "./preparations/data/outdata/"

# get a list of paths to the files
dirs = os.listdir(path_folder)
path_files = [os.path.join(path_folder, x) for x in dirs]

# Number of threads to use
num_threads = min(len(path_files), 4)  # Adjust the number of threads as needed

borders = []

# Use ThreadPoolExecutor to parallelize the finding of borders
with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Submit tasks for each image to find borders concurrently
    futures = [executor.submit(find_borders, path) for path in path_files]

    # Collect results
    for future in tqdm(concurrent.futures.as_completed(futures), "Finding borders"):
        borders.append(future.result())

# Merge borders from multiple images
bordLeft, bordRight, bordTop, bordBottom = merge_borders(borders)

print(bordLeft, bordRight, bordTop, bordBottom)

horizontal = bordRight - bordLeft
vertical = bordBottom - bordTop

print("Initial shape: ({0}, {1})".format(vertical, horizontal))

horizontalExtra = horizontal % config.WIDTH
verticalExtra = vertical % config.HEIGHT

print("Extras : ({0}, {1})".format(verticalExtra, horizontalExtra))

bordLeft = int(bordLeft + horizontalExtra / 2)
bordRight = int(bordRight - horizontalExtra / 2)

bordTop = int(bordTop + verticalExtra / 2)
bordBottom = int(bordBottom - verticalExtra / 2)

horizontal = bordRight - bordLeft
vertical = bordBottom - bordTop

print("New shape: ({0}, {1})".format(vertical, horizontal))


Finding borders: 1340it [00:37, 35.30it/s]

116 1162 283 1059
Initial shape: (776, 1046)
Extras : (8, 22)
New shape: (768, 1024)


In [6]:
def cut_data(in_path1, in_path2, out_path1, out_path2, borders):
    # Extracting border coordinates
    bordTop, bordLeft = borders
    # Starting index for saving images
    ind = 1000000
    # List of files in input paths
    files_in1 = os.listdir(in_path1)
    files_in2 = os.listdir(in_path2)
    # List to store mean intensities of processed images
    sizes = []

    # Calculating the number of splits vertically and horizontally
    vertical_split = vertical // config.HEIGHT
    horizontal_split = horizontal // config.WIDTH
         
    # Calculating grid size
    grid_size = vertical_split * horizontal_split
    
    # Iterating through each grid cell and processing images
    for i in tqdm(range(vertical_split * horizontal_split), desc="Processing"):
        # Calculating the current horizontal index within the grid
        j = i % vertical_split
        k = i // vertical_split
        # Iterating through files in the second input path
        for x in files_in2:
            # Extracting a region of interest from the first input path
            img_tmp1 = cv2.imread(os.path.join(in_path1, x), cv2.IMREAD_GRAYSCALE)[bordTop  + config.HEIGHT * k: bordTop  + config.HEIGHT * (k + 1), bordLeft + config.WIDTH * j : bordLeft + config.WIDTH * (j + 1)]
            # Calculating and storing the mean intensity of the region
            sizes.append(np.mean(img_tmp1))
    
    # Calculating the median of the mean intensities
    med = np.median(sizes)
    print(med)
    
    # Iterating through each set of images and writing selected ones
    for i in tqdm(range(len(sizes) // config.NUM_PICS - 1), "Writting images"):
        # Checking if the mean intensity of the set is greater than or equal to the median
        if np.sum(sizes[config.NUM_PICS * i : (config.NUM_PICS + 1) * i]) / config.NUM_PICS >= med:
            # Extracting indices for the selected set of images
            pic_number = [j % grid_size for j in range(config.NUM_PICS * i, (config.NUM_PICS + 1)*i - 1)]
            vertical_number = [(j // grid_size) // horizontal_split for j in range(config.NUM_PICS * i, (config.NUM_PICS + 1) * i)]
            horizontal_number = [(j // grid_size) % horizontal_split for j in range(config.NUM_PICS * i, (config.NUM_PICS + 1) * i)]

            # Iterating through images in the selected set and writing them
            for k in range(config.NUM_PICS):
                img_tmp1 = cv2.imread(os.path.join(in_path1, files_in1[pic_number[k]]), cv2.IMREAD_GRAYSCALE)[bordTop  + config.HEIGHT * vertical_number[k]: bordTop  + config.HEIGHT * (vertical_number[k] + 1), bordLeft + config.WIDTH * horizontal_number[k] : bordLeft + config.WIDTH * (horizontal_number[k] + 1)]
                img_tmp2 = cv2.imread(os.path.join(in_path2, files_in2[pic_number[k]]), cv2.IMREAD_GRAYSCALE)[bordTop  + config.HEIGHT * vertical_number[k]: bordTop  + config.HEIGHT * (vertical_number[k] + 1), bordLeft + config.WIDTH * horizontal_number[k] : bordLeft + config.WIDTH * (horizontal_number[k] + 1)]
                cv2.imwrite(os.path.join(out_path1, f"img_{ind}_{k}.png"), img_tmp1)
                cv2.imwrite(os.path.join(out_path2, f"img_{ind}_{k}.png"), img_tmp2)
            ind += 1

In [7]:
# Define the directory path
base_dir = './dataset'

# Create the 'dataset' folder
if not os.path.exists(base_dir):
    os.makedirs(base_dir)

# Create the 'indata' folder inside 'dataset'
indata_dir = os.path.join(base_dir, 'indata')
if not os.path.exists(indata_dir):
    os.makedirs(indata_dir)

# Create the 'outdata' folder inside 'dataset'
outdata_dir = os.path.join(base_dir, 'outdata')
if not os.path.exists(outdata_dir):
    os.makedirs(outdata_dir)

path_folder1 = "./preparations/data/outdata/"
path_folder1_cut = "./dataset/outdata/"

path_folder2 = "./preparations/data/indata/"
path_folder2_cut = "./dataset/indata/"

cut_data(path_folder1, path_folder2, path_folder1_cut, path_folder2_cut, (bordTop, bordLeft))


Processing: 100%|██████████| 192/192 [39:36<00:00, 12.38s/it]


0.0


Writting images:   0%|          | 0/64319 [00:00<?, ?it/s]


IndexError: list index out of range